In [12]:
import io
import numpy as np
from pymrmr import mRMR
import pandas as pd
import requests as rq

# use their test case as an example

In [15]:


data_url = 'http://home.penglab.com/proj/mRMR/test_colon_s3.csv'
data = rq.get(data_url).content
data = pd.read_csv(io.StringIO(data.decode('utf-8')))

feats = {
  10: set(['v765', 'v1123', 'v1772', 'v286', 'v467', 'v377', 'v513',
    'v1325', 'v1972', 'v1412']),
  20: set(['v765', 'v1123', 'v1772', 'v286', 'v467', 'v377', 'v513',
    'v1325', 'v1972', 'v1412', 'v1381', 'v897', 'v1671', 'v1582', 'v1423',
    'v317', 'v249', 'v1473', 'v1346', 'v125'])}

for n in feats.keys():
  selected_feats = mRMR(data, 'MIQ', n)
  assert feats[n] == set(selected_feats)